In [1]:
import time
import json

import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('oodn_odin_mcd_results_v1.csv')

In [3]:
dataset_type = 'cifar'
postprocessors = ['mcd', 'odin']
activations = ['relu', 'softplus']
optimizers = ['SGD', 'Adam']
metric = 'AUROC'

stats_for_pp = {}
for pp in postprocessors:
    total_conf = 0
    distribution_stats_per_a_o = {}
    sub_df = df[(df['Dataset'] == dataset_type) & (df['PostProcessor'] == pp)]
    for a in activations:
        for o in optimizers:
            key = a+"_"+o
            
            sub_sub_df = sub_df[(sub_df['Activation'] == a) & (sub_df['Optimizer'] == o)][metric]
            conf =  1/np.sqrt(sub_sub_df.var())
            distribution_stats_per_a_o[key] = {'mean': sub_sub_df.mean(), 
                                                  'var': sub_sub_df.var(), 
                                                  'conf': conf}    
            total_conf += conf
    
    for key, stats in distribution_stats_per_a_o.items():
        stats['weight'] = stats['conf']/total_conf
    stats_for_pp[pp] = distribution_stats_per_a_o


In [6]:
final_mean_var_pp = {}

for pp, a_o_stats in stats_for_pp.items():
    weighted_mean = 0
    for key, per_a_o_stats in a_o_stats.items():
        weighted_mean += (per_a_o_stats['mean'] * per_a_o_stats['weight'])
    final_mean_var_pp[pp] = { 'weighted_mean': weighted_mean }

    weighted_var = 0
    for key, per_a_o_stats in a_o_stats.items():
        weighted_var += per_a_o_stats['weight']*(per_a_o_stats['var'] + (weighted_mean - per_a_o_stats['mean'])**2)
    final_mean_var_pp[pp]['weighted_var'] = weighted_var

In [7]:
for pp, weighted_stats in final_mean_var_pp.items():
    print(pp, np.sqrt(weighted_stats['weighted_var'])/weighted_stats['weighted_mean'])

mcd 0.04288593604590558
odin 0.030402886587164866
